In [1]:
# %pip install scrapy 
# %pip install crochet 
# %pip install scrapy-selenium
# %pip install pandas
# %pip install SQLAlchemy

In [2]:
from typing import Iterable
import scrapy
from scrapy_selenium import SeleniumRequest
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

# Your existing Item definition
class ItemFormat(scrapy.Item):
    card_name = scrapy.Field()
    creature_type = scrapy.Field()
    mana_cost = scrapy.Field()
    card_text = scrapy.Field()
    legendary = scrapy.Field()
    card_type = scrapy.Field()
    card_rarity = scrapy.Field()
    card_set = scrapy.Field()
    card_effect_cost = scrapy.Field()

# Database setup
Base = declarative_base()

class CardData(Base):
    __tablename__ = 'card_data'

    id = Column(Integer, primary_key=True)
    card_name = Column(String)
    creature_type = Column(String)
    mana_cost = Column(String)
    card_text = Column(String)
    legendary = Column(Boolean)
    card_type = Column(String)
    card_rarity = Column(String)
    card_set = Column(String)
    
engine = create_engine('sqlite:///mtg_cards.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

# Database Pipeline
class DatabasePipeline:
    def __init__(self):
        self.session = Session()

    def process_item(self, item, spider):
        try:
            card = CardData(
                card_name = item['card_name'],
                creature_type = item.get('creature_type', ''),
                mana_cost = ','.join(item['mana_cost']),
                card_text = '\n'.join(item['card_text']),
                legendary = item.get('legendary', False),
                card_type = item.get('card_type', ''),
                card_set = item.get('card_set', '')
            )
            self.session.add(card)
            self.session.commit()
        except SQLAlchemyError as e:
            self.session.rollback()
            spider.logger.error(f"Error processing item {item['card_name']}: {str(e)}")
        return item

    def close_spider(self, spider):
        self.session.close()

def close_database_connection():
    Session.close_all()  # Close all sessions
    engine.dispose()   

# Your existing spider code
def generate_urls(max_pages):
    urls = []
    for page_number in range(1, max_pages + 1):
        base_url = 'https://gatherer.wizards.com/Pages/Search/Default.aspx?color=|[W]|[U]|[B]|[R]|[G]'
        page_url = f'https://gatherer.wizards.com/Pages/Search/Default.aspx?page={page_number}&color=|[W]|[U]|[B]|[R]|[G]'
        full_url = page_url
        urls.append(full_url)
    return urls

class DatabaseSpider(scrapy.Spider):
    name = "databaseSpider"
    custom_settings = {
        'FEED_FORMAT': 'json',
        'FEED_URI': 'databaseSpider.json',
        'ITEM_PIPELINES': {
            '__main__.DatabasePipeline': 300
        }
    }

    def start_requests(self) -> Iterable[scrapy.Request]:
        base_url = ['https://gatherer.wizards.com/Pages/Search/Default.aspx?page=255&color=|[W]|[U]|[B]|[R]|[G]']
        for url in base_url:
            yield SeleniumRequest(url=url, callback=self.parse)

    def parse(self, response):
        for i, card in enumerate(response.css('tr.cardItem.oddItem')):
            type_check = card.css('div.cardInfo span.typeLine::text').get(i)
            card_item = ItemFormat()
           
            card_item['card_name'] = card.css('div.cardInfo span.cardTitle a::text').get(i)
            if "Creature" in type_check:
                if "\u2014" in card.css('div.cardInfo span.typeLine::text').get(i):
                    text = card.css('div.cardInfo span.typeLine::text').get(i).replace("\u2014"," ").replace("                                 "," ").replace("\r\n"," ").replace("                                 "," ").replace("Creature", " ").replace("      "," ")
                    card_item["card_type"] = "Creature"
                    card_item['creature_type'] = text
                    if "Legendary" in text:
                        card_item['legendary'] = True
                    else:
                        card_item['legendary'] = False
                else:
                    card_item['card_type'] = card.css('div.cardInfo span.typeLine::text').get(i).replace("                                 "," ").replace("\r\n"," ").replace("                                 "," ")
            elif "Enchantment" in type_check:
                card_item['card_type'] = "Enchantment"
            elif "Aura" in type_check:
                card_item['card_type'] = "Aura"
            elif "Instant" in type_check:
                card_item['card_type'] = "Instant"
            elif "Battle" in type_check:
                card_item['card_type'] = "Battle"
            elif "Sorcery" in type_check:
                card_item['card_type'] = "Sorcery"
            
            card_text = card.css('div.rulesText p::text').getall()
            card_item['card_text'] = [text.replace("\u2024", "--\n").replace("\u2022", "-") for text in card_text]
            card_item['card_effect_cost'] = card.css('.rulesText img::attr(alt)').getall()
            card_item['mana_cost'] = card.css('.manaCost img::attr(alt)').getall()
            card_item['card_set'] = card.css('.rightCol img::attr(alt)').getall()

           
            yield card_item

C:\Users\ajosi\AppData\Local\Temp\ipykernel_13684\2043843322.py:22: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
    
process.crawl(DatabaseSpider)
process.start()

close_database_connection()


2024-09-12 12:24:48 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-09-12 12:24:48 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.0, Twisted 24.7.0, Python 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform Windows-10-10.0.19045-SP0
2024-09-12 12:24:48 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-12 12:24:48 [py.warnings] WARNING: c:\Python312\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FIN

In [4]:
# import pandas

# file_name = f'databaseSpider.json'
# dfjson = pandas.read_json(file_name)
# dfjson

%load_ext autoreload
%autoreload 2